In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

df = pd.read_csv("../data/ground_station_clean.csv")

In [3]:
encoders = {
    "cyclic":{
        "past":["month","dayofyear","day","hour","minute"],
        "future":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()
}

In [4]:
class PatchedPruningCallback(optuna.integration.PyTorchLightningPruningCallback, Callback):
    pass
# Criar diretório para salvar os resultados das iterações
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    num_target_lags = trial.suggest_int('num_target_lags', 5, 24)
    target_lags_list = [-i for i in range(1, num_target_lags + 1)]
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
        'gamma': trial.suggest_float('gamma', 0, 0.3, step=0.05),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 0.5, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 0.5, log=True),
        'random_state': 42, # Para reprodutibilidade
        'n_jobs': -1 # Usar todos os processadores
    } 

    pruner = PatchedPruningCallback(trial, monitor="val_loss")
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [pruner, early_stopper],
    }
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nIniciando Trial {trial.number}")
    print(f"Hiperparâmetros: {trial.params}")
    print("\nTreinando o modelo...")
    print(f"Train set: {len(train_fold)} samples") # Usar len() para TimeSeries
    print(f"Validation set: {len(val_fold)} samples") # Usar len() para TimeSeries
    
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Definir work_dir e model_name para consistência
    # force_reset=True irá limpar work_dir/model_name antes de cada treinamento da iteração.
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/" # Certifique-se que este caminho existe e é gravável
    _model_name = "model_optuna_temp" # Nome do modelo temporário para cada iteração

    # Criar o diretório _work_dir se não existir
    os.makedirs(_work_dir, exist_ok=True)
    model = XGBModel(
        model_name=_model_name,
        work_dir=_work_dir,
        lags=target_lags_list,
        output_chunk_length=1,
        pl_trainer_kwargs=pl_trainer_kwargs,
        add_encoders=encoders,
        **xgb_params
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True,
    )
    


    forecasts = model.historical_forecasts(
        train_scaled,# Como no script original
        start=0.8, # Início da previsão no conjunto de treinamento
        forecast_horizon=1,
        stride=1, # Gera uma previsão em cada ponto de tempo possível
        retrain=False,
        verbose=True # Pode ser False
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]
    metrics = {}
    print("Iniciando a verificação das séries temporais para constância...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nMétricas de performance:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_smape_val = smape(s, forecasts_t)
    print(f"O SMAPE desse fold foi {overall_smape_val}")

    trial_dict = {
        "trial_number": trial.number,
        "overall_smape_val": overall_smape_val,
        "xgb_params": xgb_params,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Resultados do trial {trial.number} salvos em {json_path}")
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")



In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-17 18:57:06,497] A new study created in memory with name: no-name-5f44bd32-0e83-4781-91f4-1ceb247f6cc6
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.



Iniciando Trial 0
Hiperparâmetros: {'num_target_lags': 21, 'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.014848633243603302, 'min_child_weight': 8, 'subsample': 0.8, 'colsample_bytree': 0.6, 'gamma': 0.2, 'reg_alpha': 0.3500648389335338, 'reg_lambda': 4.4588903127836216e-05}

Treinando o modelo...
Train set: 6647 samples
Validation set: 1662 samples
[0]	validation_0-rmse:0.35759
[1]	validation_0-rmse:0.35335
[2]	validation_0-rmse:0.34909
[3]	validation_0-rmse:0.34489
[4]	validation_0-rmse:0.34077
[5]	validation_0-rmse:0.33701


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [18:57:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "model_name", "pl_trainer_kwargs", "work_dir" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[6]	validation_0-rmse:0.33314
[7]	validation_0-rmse:0.32956
[8]	validation_0-rmse:0.32570
[9]	validation_0-rmse:0.32202
[10]	validation_0-rmse:0.31838
[11]	validation_0-rmse:0.31473
[12]	validation_0-rmse:0.31143
[13]	validation_0-rmse:0.30785
[14]	validation_0-rmse:0.30452
[15]	validation_0-rmse:0.30109
[16]	validation_0-rmse:0.29790
[17]	validation_0-rmse:0.29478
[18]	validation_0-rmse:0.29169
[19]	validation_0-rmse:0.28852
[20]	validation_0-rmse:0.28576
[21]	validation_0-rmse:0.28279
[22]	validation_0-rmse:0.27983
[23]	validation_0-rmse:0.27684
[24]	validation_0-rmse:0.27412
[25]	validation_0-rmse:0.27153
[26]	validation_0-rmse:0.26885
[27]	validation_0-rmse:0.26629
[28]	validation_0-rmse:0.26367
[29]	validation_0-rmse:0.26121
[30]	validation_0-rmse:0.25883
[31]	validation_0-rmse:0.25631
[32]	validation_0-rmse:0.25378
[33]	validation_0-rmse:0.25149
[34]	validation_0-rmse:0.24927
[35]	validation_0-rmse:0.24692
[36]	validation_0-rmse:0.24477
[37]	validation_0-rmse:0.24261
[38]	validat

historical forecasts:   0%|          | 0/1 [00:00<?, ?it/s]

...

[0]	validation_0-rmse:0.31513
[1]	validation_0-rmse:0.27748
[2]	validation_0-rmse:0.24779
[3]	validation_0-rmse:0.22444
[4]	validation_0-rmse:0.20744
[5]	validation_0-rmse:0.19447
[6]	validation_0-rmse:0.18492
[7]	validation_0-rmse:0.17748
[8]	validation_0-rmse:0.17255
[9]	validation_0-rmse:0.16842
[10]	validation_0-rmse:0.16592
[11]	validation_0-rmse:0.16408
[12]	validation_0-rmse:0.16250
[13]	validation_0-rmse:0.16118
[14]	validation_0-rmse:0.16063
[15]	validation_0-rmse:0.16013
[16]	validation_0-rmse:0.15975
[17]	validation_0-rmse:0.15964
[18]	validation_0-rmse:0.15946
[19]	validation_0-rmse:0.15931
[20]	validation_0-rmse:0.15925
[21]	validation_0-rmse:0.15922
[22]	validation_0-rmse:0.15932
[23]	validation_0-rmse:0.15930
[24]	validation_0-rmse:0.15923
[25]	validation_0-rmse:0.15927
[26]	validation_0-rmse:0.15933
[27]	validation_0-rmse:0.15929
[28]	validation_0-rmse:0.15930
[29]	validation_0-rmse:0.15941
[30]	validation_0-rmse:0.15940
[31]	validation_0-rmse:0.15960
[32]	validation_0-

historical forecasts:   0%|          | 0/1 [00:00<?, ?it/s]

Iniciando a verificação das séries temporais para constância...

Métricas de performance:
                                    MSE      RMSE       MAE        R2  \
Temperature                3.114741e-04  0.017649  0.013360  0.999909   
Precipitation_accumulated  4.907828e-03  0.070056  0.034594  0.994829   
Humidity                   1.954782e-03  0.044213  0.032568  0.999970   
Wind_Speed_kmh             5.310781e-06  0.002305  0.001259  0.999984   
Soil_Moisture              3.950028e+00  1.987468  1.458067  0.999995   
Soil_Temperature           4.723852e-04  0.021734  0.016143  0.999916   
Wind_Dir_Sin               6.099020e-07  0.000781  0.000464  0.999996   
Wind_Dir_Cos               4.263735e-07  0.000653  0.000356  0.999999   

                                SMAPE  
Temperature                  0.222039  
Precipitation_accumulated    0.007523  
Humidity                     0.039149  
Wind_Speed_kmh             112.996638  
Soil_Moisture                0.024709  
Soil_Tempera

[I 2025-05-17 19:01:58,487] Trial 49 finished with value: 28.132481233105523 and parameters: {'num_target_lags': 16, 'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.17026371737320856, 'min_child_weight': 1, 'subsample': 0.7, 'colsample_bytree': 0.6, 'gamma': 0.0, 'reg_alpha': 1.0130200568549804e-05, 'reg_lambda': 0.2846100405536689}. Best is trial 49 with value: 28.132481233105523.


Resultados do trial 49 salvos em optuna_iteration_metrics/trial_49.json
Current value: 28.132481233105523, Current params: {'num_target_lags': 16, 'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.17026371737320856, 'min_child_weight': 1, 'subsample': 0.7, 'colsample_bytree': 0.6, 'gamma': 0.0, 'reg_alpha': 1.0130200568549804e-05, 'reg_lambda': 0.2846100405536689}
Best value: 28.132481233105523, Best params: {'num_target_lags': 16, 'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.17026371737320856, 'min_child_weight': 1, 'subsample': 0.7, 'colsample_bytree': 0.6, 'gamma': 0.0, 'reg_alpha': 1.0130200568549804e-05, 'reg_lambda': 0.2846100405536689}


In [8]:
print(f"Melhor Valor (Mínimo): {study.best_value}")
print(f"Melhores Parâmetros: {study.best_params}")

Melhor Valor (Mínimo): 28.132481233105523
Melhores Parâmetros: {'num_target_lags': 16, 'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.17026371737320856, 'min_child_weight': 1, 'subsample': 0.7, 'colsample_bytree': 0.6, 'gamma': 0.0, 'reg_alpha': 1.0130200568549804e-05, 'reg_lambda': 0.2846100405536689}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Melhores resultados salvos em {json_path}")

Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
